In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import Dependencies
%matplotlib inline

# Start Python Imports
import math, time, random, datetime

# Data Manipulation
import numpy as np
import pandas as pd

# Visualization 
import matplotlib.pyplot as plt
import missingno
import seaborn as sns
plt.style.use('seaborn-whitegrid')

# Preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, label_binarize

# Machine learning
import catboost
from sklearn.model_selection import train_test_split
from sklearn import model_selection, tree, preprocessing, metrics, linear_model
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier, Pool, cv

# Let's be rebels and ignore warnings for now
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')
gender_submission = pd.read_csv('../input/titanic/gender_submission.csv') # example of what a submission should look like

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
train.isna().sum()

In [ ]:
train.dtypes

**EDA****

In [ ]:
train.Survived.value_counts()

In [ ]:

sns.countplot(data=train,x='Survived')

Fewer propotion of people survived than died.


In [ ]:
train.dtypes

## Exploring Pclass

In [ ]:
sns.countplot(data=train,x='Pclass')

In [ ]:
train["Pclass"]=train["Pclass"].astype(object)

Most people travelled in 3rd class

## Exploring Gender

In [ ]:
sns.countplot(data=train,x='Sex')

> > Most passengers are between the ages 20-50

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train["Sex"] = le.fit_transform(train["Sex"])


In [ ]:
sns.countplot(data=train,x='Sex')

## Age

In [ ]:
plt.hist(data=train,x="Age")

Most passengers are between the ages of 20-40

## Sibsp

In [ ]:
sns.countplot(data=train,x='SibSp')

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train["SibSp"] = le.fit_transform(train["SibSp"])

## Parch

In [ ]:
sns.countplot(data=train,x='Parch')

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train["Parch"] = le.fit_transform(train["Parch"])

## Ticket

In [ ]:
## This looks pointless so we drop it
train = train.drop(columns=["Ticket"])


In [ ]:
train = train.drop(columns=["Name"])

## Cabin

In [ ]:
sns.countplot(data=train,x='Cabin')

In [ ]:
#Too many missing values so we drop it
train = train.drop(columns=["Cabin"])

## Embarked

In [ ]:
sns.countplot(y='Embarked', data=train);


In [ ]:
train.dtypes

In [ ]:
cols=["Pclass","Embarked"]
train = pd.get_dummies(train, columns=cols)


In [ ]:
train = train.drop(columns=["Fare"])

In [ ]:
train.head()

## Machine learning models

In [ ]:
df = train

In [ ]:
df['Age'].fillna(df['Age'].mean(), inplace = True)


In [ ]:
X_train = df.drop('Survived', axis=1) 
y_train = df.Survived

In [ ]:
X_train.shape


In [ ]:
X_train.isnull().sum()

In [ ]:
y_train.shape

In [ ]:
def fit_ml_algo(algo, X_train, y_train, cv):
    
    # One Pass
    model = algo.fit(X_train, y_train)
    acc = round(model.score(X_train, y_train) * 100, 2)
    
    # Cross Validation 
    train_pred = model_selection.cross_val_predict(algo, 
                                                  X_train, 
                                                  y_train, 
                                                  cv=cv, 
                                                  n_jobs = -1)
    # Cross-validation accuracy metric
    acc_cv = round(metrics.accuracy_score(y_train, train_pred) * 100, 2)
    
    return train_pred, acc, acc_cv

## Run different models

In [ ]:
train_pred_log, acc_log, acc_cv_log = fit_ml_algo(LogisticRegression(), 
                                                               X_train, 
                                                               y_train, 
                                                                    10)
print("Accuracy: %s" % acc_log)
print("Accuracy CV 10-Fold: %s" % acc_cv_log)

In [ ]:
train_pred_knn, acc_knn, acc_cv_knn = fit_ml_algo(KNeighborsClassifier(), 
                                                  X_train, 
                                                  y_train, 
                                                  10)

print("Accuracy: %s" % acc_knn)
print("Accuracy CV 10-Fold: %s" % acc_cv_knn)

In [ ]:
train_pred_gaussian, acc_gaussian, acc_cv_gaussian = fit_ml_algo(GaussianNB(), 
                                                                      X_train, 
                                                                      y_train, 
                                                                           10)

print("Accuracy: %s" % acc_gaussian)
print("Accuracy CV 10-Fold: %s" % acc_cv_gaussian)

In [ ]:
train_pred_svc, acc_linear_svc, acc_cv_linear_svc = fit_ml_algo(LinearSVC(),
                                                                X_train, 
                                                                y_train, 
                                                                10)
print("Accuracy: %s" % acc_linear_svc)
print("Accuracy CV 10-Fold: %s" % acc_cv_linear_svc)

In [ ]:
train_pred_dt, acc_dt, acc_cv_dt = fit_ml_algo(DecisionTreeClassifier(), 
                                                                X_train, 
                                                                y_train,
                                                                10)

print("Accuracy: %s" % acc_dt)
print("Accuracy CV 10-Fold: %s" % acc_cv_dt)

In [ ]:
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train,y_train)

## Feature importance

In [ ]:
def feature_importance(model, data):
    """
    Function to show which features are most important in the model.
    ::param_model:: Which model to use?
    ::param_data:: What data to use?
    """
    fea_imp = pd.DataFrame({'imp': model.feature_importances_, 'col': data.columns})
    fea_imp = fea_imp.sort_values(['imp', 'col'], ascending=[True, False]).iloc[-30:]
    _ = fea_imp.plot(kind='barh', x='col', y='imp', figsize=(20, 10))
    return fea_imp

In [ ]:
feature_importance(dt_model, X_train)


## Metrics

In [ ]:
X_train.head()

In [ ]:
test.head()

In [ ]:
test1 = test

In [ ]:
test1.drop(columns=["Name","Ticket","Fare","Cabin"],inplace=True)

In [ ]:
test1["Pclass"]=test1["Pclass"].astype(object)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
test1["Sex"] = le.fit_transform(test1["Sex"])

In [ ]:
test1.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
test1["SibSp"] = le.fit_transform(test1["SibSp"])

In [ ]:
cols=["Pclass","Embarked"]
test1 = pd.get_dummies(test1, columns=cols)

In [ ]:
test1.head()

In [ ]:
wanted_test_columns = X_train.columns
wanted_test_columns


In [ ]:
test1['Age'].fillna(test1['Age'].mean(), inplace = True)


In [ ]:
predictions = dt_model.predict(test1[wanted_test_columns])

In [ ]:
submission = pd.DataFrame()
submission['PassengerId'] = test['PassengerId']
submission['Survived'] = predictions # our model predictions on the test dataset
submission.head()

In [ ]:
if len(submission) == len(test):
    print("Submission dataframe is the same length as test ({} rows).".format(len(submission)))
else:
    print("Dataframes mismatched, won't be able to submit to Kaggle.")


In [ ]:
submission.to_csv('..decisontree_submission.csv', index=False)
print('Submission CSV is ready!')

In [ ]:
#submissions_check = pd.read_csv("../decisontree_submission.csv")
#submissions_check.head()

In [ ]:
filename = 'Titanic Predictions 1.csv'

submission.to_csv(filename,index=False)

print('Saved file: ' + filename)

## ANN

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train1 = sc.fit_transform(X_train)


In [ ]:
import tensorflow as tf

In [ ]:
ann = tf.keras.models.Sequential()

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [ ]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
ann.fit(X_train1, y_train, batch_size = 32, epochs = 100)

In [ ]:
X_test = sc.transform(test1[wanted_test_columns])

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
y_pred = y_pred.astype(int)

In [ ]:
submission1 = pd.DataFrame()
submission1['PassengerId'] = test['PassengerId']
submission1['Survived'] = y_pred # our model predictions on the test dataset
submission1.head()

In [ ]:
if len(submission1) == len(test):
    print("Submission dataframe is the same length as test ({} rows).".format(len(submission)))
else:
    print("Dataframes mismatched, won't be able to submit to Kaggle.")

In [ ]:
filename = 'Titanic Predictions ANN.csv'

submission1.to_csv(filename,index=False)

print('Saved file: ' + filename)